In [8]:
from datetime import datetime, timedelta, timezone
import threading
import time
import os

import pandas as pd
import requests

from mainnet_launch.database.schema.full import Blocks
from mainnet_launch.database.postgres_operations import (
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
    _exec_sql_and_cache,
    get_full_table_as_df
)
from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks, build_blocks_to_use

from mainnet_launch.constants import *
from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama


df = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == SONIC_CHAIN.chain_id).set_index('datetime').sort_index()
df['found_timestamp'] = df.index

daily_df = df.resample('1d')[['found_timestamp']].last()

In [9]:
import plotly.express as px
daily_df['diff'] = (daily_df['found_timestamp'].diff().dt.total_seconds() / 3600) - 24
px.scatter(daily_df, x=daily_df.index, y='diff').show()